In [1]:
import pandas as pd
import numpy as np

In [2]:
#load dataframe for NAICS code to description
naics17 = pd.DataFrame(pd.read_csv('data/naics17.txt', sep=","))
naics12 = pd.DataFrame(pd.read_csv('data/naics12.txt', sep=","))
naics08 = pd.DataFrame(pd.read_csv('data/naics08.csv'))
naics03 = pd.DataFrame(pd.read_csv('data/naics03.csv'))
naics98 = pd.DataFrame(pd.read_csv('data/naics98.csv'))

#clean dataframes
naics12['DESCRIPTION'] = list(map(lambda x: x[:-1], naics12['DESCRIPTION']))

#turn dataframe into dictionary
naics17 = naics17.set_index("NAICS").to_dict()['DESCRIPTION']
naics12 = naics12.set_index("NAICS").to_dict()['DESCRIPTION']
naics08 = naics08.set_index("NAICS").to_dict()['DESCRIPTION']
naics03 = naics03.set_index("NAICS").to_dict()['DESCRIPTION']
naics98 = naics98.set_index("NAICS").to_dict()['DESCRIPTION']

#import zip to FIPS, zip to County, zip to State
zipFIPS = pd.DataFrame(pd.read_csv('data/zip_FIPS.csv'), dtype=str)[["ZIP", "STCOUNTYFP"]]
zipCOUNTY = pd.DataFrame(pd.read_csv('data/zip_FIPS.csv', dtype = str))[["ZIP", "COUNTYNAME"]]
zipSTATE = pd.DataFrame(pd.read_csv('data/zip_FIPS.csv', dtype = str))[["ZIP", "STATE"]]

#fixing problematic imports
zipFIPS['STCOUNTYFP'] = list (map(lambda x: "0" + x if len(x) == 4 else x, zipFIPS['STCOUNTYFP']))

#turn dataframe into dictionary
zipFIPS = zipFIPS.set_index("ZIP").to_dict()["STCOUNTYFP"]
zipCOUNTY = zipCOUNTY.set_index("ZIP").to_dict()["COUNTYNAME"]
zipSTATE = zipSTATE.set_index("ZIP").to_dict()["STATE"]

#states = ["AL","AK","FL","GA","LA","MD","MS","NC","SC","TN","TX"]

In [3]:
def cleanDf(file, naics):
    #importing small businesses dataset, casting to int
    sb = pd.read_csv(file, sep=",", dtype = str)
    #find the yearo f the dataset
    year = int("20" + file[8:10])
    #year correction from 2098, 2099 to 1998, 1999
    if (year > 2021):
        year = year - 100
    
    #make all columns lowercase to standardize
    sb.columns = [s.lower() for s in sb.columns]

    
    #data from 1998 has incorrectly named zip column
    if (year == 1998):
        col = list(sb.columns)
        col[0] = "zip"
        sb.columns = col
        
    #standardize column names
    if ("n1_4" not in sb.columns):
        sb = sb.rename(columns = {"n<5":"n1_4"})
    
    #initial dimensions of the dataset
    print(file)
    print(sb.shape)

    #filer for desired naics label - remove for more detail
    sb = sb[sb['naics'].isin(["------"])]
    
    
    #Add FIPS for zip, remove invalid ZIP
    sb = sb[sb['zip'].isin(zipFIPS.keys())]    
    sb['FIPS'] = list(map(lambda x: zipFIPS[x], sb['zip']))
    
    #adding and filtering  state based off zip classification
    sb['state'] = list(map(lambda x: zipSTATE[x], sb['zip']))
    #sb = sb[sb['state'].isin(states)]
    
    #add counties 
    sb['county'] = list(map(lambda x: zipCOUNTY[x], sb['zip']))

    #adding corresponding industry descriptions for each naics code
    if (naics == 17):
        sb['naics_desc'] = list(map(lambda x: naics17[x], sb['naics']))
    elif (naics == 12):
        sb['naics_desc'] = list(map(lambda x: naics12[x], sb['naics']))
    elif (naics == 8):
        sb['naics_desc'] = list(map(lambda x: naics08[x], sb['naics']))
    elif (naics == 3):
        sb['naics_desc'] = list(map(lambda x: naics03[x], sb['naics']))
    elif (naics == 98):
        sb['naics_desc'] = list(map(lambda x: naics98[x], sb['naics']))
        
    #add year column
    sb['year'] = year
    
    #final dataset dim
    print(sb.shape)
    
    return sb

In [4]:
sb18 = cleanDf('data/zbp18detail.txt', 17)
sb17 = cleanDf('data/zbp17detail.txt', 17)
sb16 = cleanDf('data/zbp16detail.txt', 12)
sb15 = cleanDf('data/zbp15detail.txt', 12)
sb14 = cleanDf('data/zbp14detail.txt', 12)
sb13 = cleanDf('data/zbp13detail.txt', 12)
sb12 = cleanDf('data/zbp12detail.txt', 12)
sb11 = cleanDf('data/zbp11detail.txt', 8)
sb10 = cleanDf('data/zbp10detail.txt', 8)
sb09 = cleanDf('data/zbp09detail.txt', 8)
sb08 = cleanDf('data/zbp08detail.txt', 8)
sb07 = cleanDf('data/zbp07detail.txt', 3)
sb06 = cleanDf('data/zbp06detail.txt', 3)
sb05 = cleanDf('data/zbp05detail.txt', 3)
sb04 = cleanDf('data/zbp04detail.txt', 3)
sb03 = cleanDf('data/zbp03detail.txt', 3)
sb02 = cleanDf('data/zbp02detail.txt', 98)
sb01 = cleanDf('data/zbp01detail.txt', 98)
sb00 = cleanDf('data/zbp00detail.txt', 98)
sb99 = cleanDf('data/zbp99detail.txt', 98)
sb98 = cleanDf('data/zbp98detail.txt', 98)

data/zbp18detail.txt
(2874446, 12)
(32097, 17)
data/zbp17detail.txt
(2870579, 12)
(32123, 17)
data/zbp16detail.txt
(8418283, 12)
(34997, 17)
data/zbp15detail.txt
(8403660, 12)
(35014, 17)
data/zbp14detail.txt
(8377070, 12)
(35055, 17)
data/zbp13detail.txt
(8371007, 12)
(35057, 17)
data/zbp12detail.txt
(8380408, 12)
(35081, 17)
data/zbp11detail.txt
(3196767, 12)
(35099, 17)
data/zbp10detail.txt
(3219077, 12)
(35153, 17)
data/zbp09detail.txt
(3238421, 12)
(35205, 17)
data/zbp08detail.txt
(3290137, 12)
(35244, 17)
data/zbp07detail.txt
(3340643, 12)
(35252, 17)
data/zbp06detail.txt
(3321529, 12)
(35223, 17)
data/zbp05detail.txt
(3307690, 12)
(35192, 17)
data/zbp04detail.txt
(3296175, 12)
(35138, 17)
data/zbp03detail.txt
(3267445, 12)
(34987, 17)
data/zbp02detail.txt
(3251827, 12)
(35010, 17)
data/zbp01detail.txt
(3279975, 12)
(34988, 17)
data/zbp00detail.txt
(3260522, 12)
(34968, 17)
data/zbp99detail.txt
(3224595, 12)
(34892, 17)
data/zbp98detail.txt
(3190439, 12)
(34844, 17)


In [5]:
allYears = pd.concat([sb18, sb17, sb16, sb15, sb14, sb13, sb12, sb11, sb10,
                      sb09, sb08, sb07, sb06, sb05, sb04, sb03, sb02, sb01, sb00,
                      sb99, sb98], ignore_index=True)
allYears.head()

,zip,naics,est,n1_4,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,FIPS,state,county,naics_desc,year
0,10001,------,7248,4052,1222,820,647,254,156,62,24,11,36061,NY,New York County,Total for all sectors,2018
1,10002,------,2962,2000,444,282,172,44,15,4,N,N,36061,NY,New York County,Total for all sectors,2018
2,10003,------,4268,2551,599,497,366,137,78,25,11,4,36061,NY,New York County,Total for all sectors,2018
3,10004,------,1600,756,272,207,207,72,49,14,14,9,36061,NY,New York County,Total for all sectors,2018
4,10005,------,1346,676,210,142,144,81,66,13,11,3,36061,NY,New York County,Total for all sectors,2018


In [6]:
allYears.shape

(730619, 17)

In [7]:
#allYears.to_csv("SB_Data")
#total = pd.DataFrame(pd.read_csv('SB_Data.csv', sep=","))
#total = total[total.columns[1:]]

In [8]:
#wholeSectors = total[total['naics'] == "------"]

In [9]:
allYears["est"] = pd.to_numeric(allYears["est"], errors = "coerce")
allYears["n1_4"] = pd.to_numeric(allYears["n1_4"], errors = "coerce")
allYears["n5_9"] = pd.to_numeric(allYears["n5_9"], errors = "coerce")
allYears["n10_19"] = pd.to_numeric(allYears["n10_19"], errors = "coerce")
allYears["n20_49"] = pd.to_numeric(allYears["n20_49"], errors = "coerce")
allYears["n50_99"] = pd.to_numeric(allYears["n50_99"], errors = "coerce")
allYears["n100_249"] = pd.to_numeric(allYears["n100_249"], errors = "coerce")
allYears["n250_499"] = pd.to_numeric(allYears["n250_499"], errors = "coerce")
allYears["n500_999"] = pd.to_numeric(allYears["n500_999"], errors = "coerce")
allYears["n1000"] = pd.to_numeric(allYears["n1000"], errors = "coerce")

In [11]:
allYears.head()

,zip,naics,est,n1_4,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,FIPS,state,county,naics_desc,year,n1_19
0,10001,------,7248,4052.0,1222.0,820.0,647.0,254.0,156.0,62.0,24.0,11.0,36061,NY,New York County,Total for all sectors,2018,6094.0
1,10002,------,2962,2000.0,444.0,282.0,172.0,44.0,15.0,4.0,NaN,NaN,36061,NY,New York County,Total for all sectors,2018,2726.0
2,10003,------,4268,2551.0,599.0,497.0,366.0,137.0,78.0,25.0,11.0,4.0,36061,NY,New York County,Total for all sectors,2018,3647.0
3,10004,------,1600,756.0,272.0,207.0,207.0,72.0,49.0,14.0,14.0,9.0,36061,NY,New York County,Total for all sectors,2018,1235.0
4,10005,------,1346,676.0,210.0,142.0,144.0,81.0,66.0,13.0,11.0,3.0,36061,NY,New York County,Total for all sectors,2018,1028.0


In [12]:
allYearsTest = allYears
allYearsTest['year'] = allYearsTest['year'] -1

In [21]:
groupedSectors = allYearsTest.groupby(["FIPS","state","county","year"])["est", "n1_4","n5_9","n10_19","n1_19","n20_49","n50_99","n100_249","n250_499","n500_999","n1000"].sum()

#groupedSectors = allYears.groupby(["FIPS","state","county","year"])["est", "n1_4","n5_9","n10_19","n20_49","n50_99","n100_249","n250_499","n500_999","n1000"].sum()
groupedSectors

est   n1_4  n5_9  n10_19  n1_19  n20_49  \
FIPS  state county         year                                            
01001 AL    Autauga County 1997   67   41.0  11.0    12.0   64.0     2.0   
                           1998   74   47.0  12.0    11.0   70.0     3.0   
                           1999   70   47.0  12.0     7.0   66.0     3.0   
                           2000   83   62.0  14.0     4.0   80.0     2.0   
                           2001   48   31.0  10.0     4.0   45.0     2.0   
...                              ...    ...   ...     ...    ...     ...   
56045 WY    Weston County  2013  322  195.0  66.0    37.0  298.0    17.0   
                           2014  321  189.0  72.0    38.0  299.0    14.0   
                           2015  333  212.0  60.0    38.0  310.0    17.0   
                           2016  331  208.0  56.0    45.0  309.0    16.0   
                           2017  321  204.0  54.0    39.0  297.0    18.0   

                                 n50_99  n100_249  n250_499  n500_999  n1000  
FIPS  state county         year                                               
01001 AL    Autauga County 1997     0.0       1.0       0.0       0.0    0.0  
                           1998     0.0       1.0       0.0       0.0    0.0  
                           1999     0.0       1.0       0.0       0.0    0.0  
                           2000     0.0       1.0       0.0       0.0    0.0  
                           2001     1.0       0.0       0.0       0.0    0.0  
...                                 ...       ...       ...       ...    ...  
56045 WY    Weston County  2013     4.0       3.0       0.0       0.0    0.0  
                           2014     5.0       3.0       0.0       0.0    0.0  
                           2015     4.0       2.0       0.0       0.0    0.0  
                           2016     0.0       0.0       0.0       0.0    0.0  
                           2017     0.0       3.0       0.0       0.0    0.0  

[62741 rows x 11 columns]

In [22]:
import itertools
counties = np.unique(list(map(lambda x: x[0], groupedSectors.index)))

In [23]:
def createDifferences(f, col, n):
        df0 = groupedSectors.loc[f]
        res0 = 100 * np.diff(df0[col], n)/df0[col][:-n]
        result = list(itertools.chain([np.nan] * n, res0))
        return result
    
def createTimeDif(county, column):
    listLoanChange = [createDifferences(f, column, 1) for f in county]
    return listLoanChange

In [24]:
import itertools
def createDifferences2(df, f, col, n):
        df0 = df.loc[f]
        res0 = np.diff(df0[col], n)
        result = list(itertools.chain([np.nan] * n, res0))
        return result
    
def createTimeDif2(df, county, column):
    listLoanChange = [createDifferences2(df, f, column, 1) for f in county]
    return listLoanChange

In [25]:
groupedSectors[groupedSectors.columns[1:]] = groupedSectors[groupedSectors.columns[1:]].replace(0, np.nan)

In [38]:
groupedSectors["n1_19"] = groupedSectors["n1_4"] + groupedSectors["n5_9"] + groupedSectors["n10_19"]

In [39]:
groupedSectors['businessChange'] = list(itertools.chain.from_iterable(createTimeDif(counties, "est")))
groupedSectors['businessChange2'] = list(itertools.chain.from_iterable(createTimeDif2(groupedSectors, counties, "est")))
groupedSectors['onefourChange'] = list(itertools.chain.from_iterable(createTimeDif(counties, "n1_4")))
groupedSectors['fivenineChange'] = list(itertools.chain.from_iterable(createTimeDif(counties, "n5_9")))
groupedSectors['onenineteenChange2'] = list(itertools.chain.from_iterable(createTimeDif2(groupedSectors, counties, "n1_19")))

In [40]:
def createMVA(n, f, col):
    window_size = n
    df0 = groupedSectors.loc[f][col]
    numbers_series = pd.Series(df0)
    windows = numbers_series.rolling(window_size)
    moving_averages = windows.mean()
    moving_averages_list = moving_averages.tolist()
    return moving_averages_list

def createMVAs(n, county, col):
    listLoanChange = [createMVA(n, f, col) for f in county]
    return listLoanChange

groupedSectors['businessChangeMVA3'] = list(itertools.chain.from_iterable(createMVAs(3, counties, "businessChange")))
groupedSectors['businessChangeMVA5'] = list(itertools.chain.from_iterable(createMVAs(5, counties, "businessChange")))
groupedSectors['estMVA3'] = list(itertools.chain.from_iterable(createMVAs(3, counties, "est")))
groupedSectors['estMVA5'] = list(itertools.chain.from_iterable(createMVAs(5, counties, "est")))

In [41]:
pd.Series(groupedSectors['n1_19'] - groupedSectors['n1_4'] - groupedSectors['n5_9'] - groupedSectors['n10_19']).value_counts()

0.0    60571
dtype: int64

In [42]:
#groupedSectors.to_csv("smallbiz.csv")
groupedSectors.to_csv("smallbiz1.csv")

In [ ]:
#loan related to income
#loan related to population